In [5]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


train_df = pd.read_csv('./origin_data/train_clean.csv')
MLM_values = train_df['MLM'].values
HLM_values = train_df['HLM'].values
train_df = train_df.drop(columns=['id','SMILES','MLM','HLM']).values
# 물리화학적 특징 (예시)
physchem_features = torch.tensor(train_df)  # 이 부분은 데이터에 따라 다를 수 있습니다.



# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(physchem_features, MLM_values, test_size=0.2)

# 2. 트랜스포머 모델 구성
class MLM_Predictor(nn.Module):
    def __init__(self):
        super(MLM_Predictor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        return self.fc(pooled_output)

model = MLM_Predictor()

# 3. 모델 훈련
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.MSELoss()

for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train['input_ids'], X_train['attention_mask'])
    loss = torch.sqrt(loss_fn(outputs, y_train))
    loss.backward()
    optimizer.step()

# 4. 예측
model.eval()
with torch.no_grad():
    predictions = model(X_test['input_ids'], X_test['attention_mask'])


IndexError: too many indices for tensor of dimension 2